In [3]:
# --- IMPORTS ---
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

# --- LOAD MODEL FROM JSON + WEIGHTS ---
with open("saved_models/model_architecture.json", "r") as json_file:
    model_json = json_file.read()

model = model_from_json(model_json)
model.load_weights("saved_models/model_weights.weights.h5")

print(" Model loaded successfully for prediction.")

# --- PREPARE TEST DATA ---
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "dataset/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# --- PREDICTION ---
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

# --- ACCURACY & CONFUSION MATRIX ---
accuracy = accuracy_score(true_labels, predicted_classes)
cm = confusion_matrix(true_labels, predicted_classes)

print(f"\n Test Accuracy: {accuracy * 100:.2f}%")
print("\n Confusion Matrix:")
print(cm)


 Model loaded successfully for prediction.
Found 180 images belonging to 12 classes.


c:\Users\timde\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step

 Test Accuracy: 13.89%

 Confusion Matrix:
[[ 0  0  0  0  0  4  3  3  0  0  0  5]
 [ 0  0  0  0  0 10  2  2  0  0  0  1]
 [ 0  0  0  0  0  1  8  0  0  0  0  6]
 [ 0  0  0  0  0  3  2  3  1  0  0  6]
 [ 0  0  0  0  0  0  5  6  0  0  0  4]
 [ 0  0  0  0  0  7  6  1  0  0  0  1]
 [ 0  0  0  0  0  4 10  0  0  0  0  1]
 [ 0  0  0  0  0  2 10  3  0  0  0  0]
 [ 0  0  0  0  0  6  2  0  3  0  1  3]
 [ 0  0  0  0  1  2  7  2  1  0  0  2]
 [ 0  0  0  0  0  5  1  2  3  0  0  4]
 [ 0  0  0  0  0  6  7  0  0  0  0  2]]


In [24]:
# --- FULL NEW CELL: RANDOM 5 TEST IMAGES + CORRECT COUNT ---

import random
import numpy as np

# --- YOUR CONDITION DESCRIPTIONS ---
nail_condition_rules = {
    "Bluish Nail": "Possible oxygen deficiency or cyanosis; may indicate heart or lung problems.",
    "Clubbing": "Possible chronic lung or heart disease causing low oxygen levels.",
    "Onychomycosis": "Possible fungal nail infection; causes thick, brittle, or discolored nails.",
    "Psoriasis": "Possible autoimmune condition causing pitting or discoloration on nails.",
    "Healthy Nail": "Normal nail appearance; no visible abnormalities or diseases.",
    "Acral Lentiginous Melanoma": "Possible serious skin cancer forming under the nail; seek medical attention.",
    "Onychogryphosis": "Possible nail thickening or curvature; common in elderly or due to trauma.",
    "Pitting": "Possible psoriasis or eczema; small depressions or dents on the nail surface.",
    "Yellow Nail": "Possible fungal infection, liver disease, or chronic respiratory issue.",
    "White Nail": "Possible liver disease, low protein levels, or anemia.",
    "Beau’s Lines": "Possible history of illness, infection, or stress disrupting nail growth.",
    "Koilonychia": "Possible iron deficiency anemia; nail appears spoon-shaped."
}

# --- INDEX TO CLASS NAME MAPPING ---
condition_mapping = {v: k for k, v in test_generator.class_indices.items()}

# --- PICK 5 RANDOM INDEXES ---
random_indexes = random.sample(range(len(test_generator.filenames)), 5)

# --- PRINT HEADER ---
print("\n{:<75} {:<30} {:<30} {}".format(
    "FILENAME", "TRUE LABEL", "PREDICTED", "DESCRIPTION"
))
print("-" * 170)

correct_count = 0

# --- PRINT INFO FOR EACH RANDOM IMAGE ---
for i in random_indexes:
    filename = test_generator.filenames[i]
    true_name = condition_mapping[true_labels[i]]
    pred_name = condition_mapping[predicted_classes[i]]
    desc = nail_condition_rules.get(pred_name, "No description available.")

    if true_name == pred_name:
        correct_count += 1

    print("{:<75} {:<30} {:<30} {}".format(
        filename, true_name, pred_name, desc
    ))

print(f"\n✅ Correct predictions out of 5 random samples: {correct_count} / 5")
print(f"✅ Random 5 images displayed out of {len(test_generator.filenames)} total.")



FILENAME                                                                    TRUE LABEL                     PREDICTED                      DESCRIPTION
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Acral Lentiginous Melanoma\45Screenshot 2025-10-26 003109.png               Acral Lentiginous Melanoma     Onychogryphosis                Possible nail thickening or curvature; common in elderly or due to trauma.
Onychogryphosis\Screenshot 2025-10-25 211836.png                            Onychogryphosis                Onychogryphosis                Possible nail thickening or curvature; common in elderly or due to trauma.
White Nail (Leukonychia)\Screenshot 2025-10-10 at 3.43.14 PM.png            White Nail (Leukonychia)       Onychomycosis                  Possible fungal nail infection; causes thick, brittle, or discolored nails.
Healthy Nail\Screenshot 2025-10-25 224